In [129]:
%load_ext autoreload
%autoreload 2
import spike_queries
import pandas as pd
import tqdm
import pickle
import random
import itertools

import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertModel, AutoTokenizer, AutoModel, PreTrainedTokenizerFast
import numpy as np
from typing import List
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from typing import Dict, Tuple
from scipy.spatial.distance import cosine as cosine_distance
from collections import defaultdict


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [130]:
with open("queries2.txt", "r") as f:
    queries = f.readlines()
    queries = [l.strip() for l in queries]


In [131]:
for i,q in enumerate(queries):
    try:
        assert len(q.split("\t")) == 2
    except:
        print(i, q, len(q.split("\t")))

In [132]:
for i,q in enumerate(queries):
    a = q.count("arg1:") == q.count("arg2:") == 1
    if not a:
        print(i,q)

In [133]:
queries2results = defaultdict(list)
id2query = defaultdict(list)

for id_and_query in queries:
    query,id = id_and_query.split("\t")
    id2query[int(id)].append(query)
    
print(id2query[1])

['<>arg1:virus $infection $causes a <>arg2:condition', '<>arg1:virus $infection, which $causes a <>arg2:condition', '<>arg1:virus $infection $results $in <>arg2:condition', '<>arg1:virus $infection, which $results $in <>arg2:condition', '<>arg2:condition $is $the $result of <>arg1:virus $infection', '<>arg2:condition, which $is $the $result of <>arg1:virus $infection', '<>arg2:condition $is $[l=cause|trigger]caused $by <>arg1:virus $infection', 'as a result of <>arg1:virus $infection, <>arg2:condition $can $develop']


In [134]:
dataset_name = "covid19"
num_results = 125
query_type = "syntactic"
import time

for i, q_and_id in tqdm.tqdm(enumerate(queries), total = len(queries)):
    q, id = q_and_id.split("\t")
    id = int(id)
    try:
        df = spike_queries.perform_query(q, dataset_name, num_results, query_type) #previously: word=Hawaii
        queries2results[id].append(df)
        time.sleep(2)
    except Exception as e:
        print("Error", i+1)
        print(e)
        
        continue

100%|██████████| 389/389 [1:06:36<00:00, 10.27s/it]


In [135]:
# for id, dfs in queries2results.items():
#      for i,df in enumerate(dfs):
#          print(id, i)
#          queries2results[id][i] = df.dropna(subset=['arg1_first_index',"arg1_last_index","arg2_first_index","arg2_last_index"])

1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 0
2 1
2 2
3 0
3 1
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
7 0
7 1
7 2
7 3
7 4
8 0
8 1
8 2
8 3
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
12 0
12 1
12 2
13 0
13 1
13 2
13 3
13 4
14 0
14 1
14 2
14 3
14 4
14 5
14 6
15 0
15 1
15 2
15 3
15 4
15 5
15 6
15 7
15 8
15 9
15 10
15 11
15 12
16 0
16 1
16 2
16 3
16 4
16 5
16 6
16 7
16 8
17 0
17 1
17 2
17 3
18 0
18 1
18 2
18 3
18 4
18 5
19 0
19 1
19 2
19 3
19 4
19 5
19 6
20 0
20 1
20 2
20 3
20 4
21 0
21 1
21 2
21 3
21 4
21 5
22 0
22 1
22 2
22 3
22 4
22 5
23 0
23 1
23 2
23 3
23 4
23 5
23 6
23 7
23 8
23 9
23 10
24 0
24 1
24 2
25 0
25 1
25 2
25 3
26 0
26 1
26 2
27 0
27 1
27 2
28 0
28 1
28 2
29 0
29 1
29 2
29 3
29 4
29 5
29 6
30 0
30 1
30 2
30 3
30 4
31 0
31 1
31 2
31 3
31 4
31 5
31 6
32 0
32 1
32 2
32 3
32 4
32 5
32 6
33 0
33 1
33 2
33 3
34 0
34 1
34 2
35 0
35 

In [136]:
for id, dfs in queries2results.items():
      for i,df in enumerate(dfs):
            queries2results[id][i]["spike_query"] = id2query[id][i]

In [137]:
# with open("queries2results2.pickle", "wb") as f:
#       pickle.dump(queries2results, f)

In [138]:
with open("queries2results2.pickle", "rb") as f:
    queries2results = pickle.load(f)

In [139]:
queries2results.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57])

In [70]:
# queries2results_new = dict()
# for k,dfs in queries2results.items():
#     if k == 0: continue
#     queries2results_new[k] = pd.concat(dfs, axis=0)
    
# queries2results = queries2results_new

In [140]:
ids = set([int(l.split("\t")[-1]) for l in queries])

In [141]:
def add_arguments(sent:str, arg1_start, arg1_end, arg2_start, arg2_end):
    
        s_lst = sent.split(" ")
        if arg1_start > arg2_start:
            arg1_start, arg2_start = arg2_start, arg1_start
            arg1_end, arg2_end = arg2_end, arg1_end
            arg1_str, arg2_str = "<<ARG2:", "<<ARG1:"
        else:
            arg1_str, arg2_str = "<<ARG1:", "<<ARG2:"
        
        s_with_args = s_lst[:arg1_start] + [arg1_str] + s_lst[arg1_start:arg1_end+1] + [">>"] + s_lst[arg1_end+1:arg2_start] + [arg2_str] + s_lst[arg2_start:arg2_end+1] + [">>"] +s_lst[arg2_end+1:]  
        #s_with_args = s_lst[:arg1_start] + [arg1_str+s_lst[arg1_ind]] + s_lst[arg1_ind+1:arg2_ind] + [arg2_str+s_lst[arg2_ind]] + s_lst[arg2_ind+1:]
        s_with_args = " ".join(s_with_args).replace("ARG1: ", "ARG1:").replace("ARG2: ", "ARG2:")
        s_with_args = s_with_args.replace(" >>", ">>")
        return s_with_args
    

In [142]:
NUM_PAIRS_PER_RELATION = 1000

data = []

for id in tqdm.tqdm(ids): #foreach relation
    
    pattern_pairs = list(itertools.product(range(len(id2query[id])), repeat=2))
    pattern_pairs_different = [(p1,p2) for (p1,p2) in pattern_pairs if p1 != p2]
    pattern_pairs_same = [(p1,p2) for (p1,p2) in pattern_pairs if p1 == p2]
    pattern_pairs_different += pattern_pairs_same
    sum_pattern = 0
    relation_examples = []
    
    for p1, p2 in pattern_pairs_different:
        df1,df2 = queries2results[id][p1], queries2results[id][p2]
        if df1.empty or df2.empty: continue
            
        sentences1, sentences2 = df1["sentence_text"].tolist(), df2["sentence_text"].tolist()
        query1, query2 = df1["spike_query"].tolist()[0], df2["spike_query"].tolist()[0]
        arg1_first_start, arg1_first_end = df1["arg1_first_index"].tolist(), df1["arg1_last_index"].tolist()
        arg2_first_start, arg2_first_end = df1["arg2_first_index"].tolist(), df1["arg2_last_index"].tolist()
        arg1_second_start, arg1_second_end = df2["arg1_first_index"].tolist(), df2["arg1_last_index"].tolist()
        arg2_second_start, arg2_second_end = df2["arg2_first_index"].tolist(), df2["arg2_last_index"].tolist()
        
        
        all_pair_combinations = list(itertools.product(range(len(df1)), range(len(df2))))
        random.shuffle(all_pair_combinations)
        for combination in all_pair_combinations[:100]:
            ind1, ind2 = combination
            
            sent1, sent2 = sentences1[ind1], sentences2[ind2]
            sent1_arg1_start, sent1_arg1_end = arg1_first_start[ind1], arg1_first_end[ind1]
            sent1_arg2_start, sent1_arg2_end = arg2_first_start[ind1], arg2_first_end[ind1]
            sent2_arg1_start, sent2_arg1_end = arg1_second_start[ind2], arg1_second_end[ind2]
            sent2_arg2_start, sent2_arg2_end = arg2_second_start[ind2], arg2_second_end[ind2]
            sent1_with_args = add_arguments(sent1, sent1_arg1_start, sent1_arg1_end, sent1_arg2_start, sent1_arg2_end)
            sent2_with_args = add_arguments(sent2, sent2_arg1_start, sent2_arg1_end, sent2_arg2_start, sent2_arg2_end)
            
            sent1_lst = sent1.split(" ")
            sent2_lst = sent2.split(" ")
            
            sent1_arg1_w = sent1_lst[sent1_arg1_start:sent1_arg1_end+1]
            sent1_arg2_w = sent1_lst[sent1_arg2_start:sent1_arg2_end+1]
            sent2_arg1_w = sent2_lst[sent2_arg1_start:sent2_arg1_end+1]
            sent2_arg2_w = sent2_lst[sent2_arg2_start:sent2_arg2_end+1]
            
            d = {"first": sent1_with_args, "second": sent2, "second_with_arguments": sent2_with_args, "query_first": query1, "query_second": query2, 
                 "first_arg1": (sent1_arg1_start, sent1_arg1_end), "first_arg2": (sent1_arg2_start, sent1_arg2_end), 
                 "second_arg1": (sent2_arg1_start, sent2_arg1_end),
                 "second_arg2": (sent2_arg2_start, sent2_arg2_end), "first_arg1_words": sent1_arg2_w,
                "first_arg2_words": sent1_arg2_w, "second_arg1_words": sent2_arg1_w, "second_arg2_words": sent2_arg2_w,
                "query_id": id}
            relation_examples.append(d)
    
    random.shuffle(relation_examples)
    relation_examples = relation_examples[:NUM_PAIRS_PER_RELATION]
    data.extend(relation_examples)
    
random.shuffle(data)

100%|██████████| 57/57 [00:16<00:00,  3.51it/s]


In [143]:
lengths = [len(d["first"].split(" ")) + len(d["second"].split(" ")) for d in data]
data = [d for i,d in enumerate(data) if lengths[i] < 250]
data = [d for d in data if (d["second_arg1"] != d["second_arg2"]) and (d["first_arg1"] != d["first_arg2"])]
data = [d for d in data if d["first_arg1_words"] != [''] and d["first_arg2_words"] != ['']
       and d["second_arg1_words"] != [''] and d["second_arg2_words"] != ['']]

def is_number_between_brackets(sent, ind):
    lst = sent.split(" ")
    if not is_number(lst[ind]): return False
    if ind == len(lst) - 1 or ind == 0: return False
    return (lst[ind -1] == "[" and lst[ind + 1] == "]") or (lst[ind -1] == "(" and lst[ind + 1] == ")")

def is_number(s):
    try:
        d = int(s)
        return True
    except:
        return False

data2 = [d for d in data if not is_number_between_brackets(d["second"], d["second_arg1"][0]) and not
       is_number_between_brackets(d["second"], d["second_arg2"][0]) and not
        is_number_between_brackets(d["first"], d["first_arg1"][0]) and not
        is_number_between_brackets(d["first"], d["first_arg2"][0])]

ids_subset = np.random.choice(list(ids), size = int(0.2 * len(ids)))
data_train = [d for d in data if d["query_id"] not in ids_subset]
data_dev = [d for d in data if d["query_id"] in ids_subset]
print(ids_subset)

[30 13 48 18 47 18 47 44 41 40 21]


In [144]:
same = [d for d in data if d["query_first"] == d["query_second"]]
not_same = [d for d in data if d["query_first"] != d["query_second"]]
print(len(same), len(not_same))

8803 39917


In [145]:
not_same[0]

{'first': '<<ARG1:Griseofulvin>> should not be used to treat <<ARG2:dermatophytosis>> , because it has been associated with bone marrow suppression in FIV-positive cats.79 Recombinant human erythropoietin or darbepoetin may be useful for some cats with nonregenerative anemia and does not appear to increase viral load through activation of transcription of latent virus.80 Although recombinant human granulocyte colony stimulating factor ( G-CSF ) can increase neutrophil counts in cats infected with FIV , antibodies can develop within a few weeks that cross-react with endogenous G-CSF .',
 'second': 'Vitamin C is contraindicated in liver disease associated with transition metal accumulation ( copper , iron ) because it may enhance oxidative injury .',
 'second_with_arguments': '<<ARG1:Vitamin C>> is contraindicated in <<ARG2:liver disease>> associated with transition metal accumulation ( copper , iron ) because it may enhance oxidative injury .',
 'query_first': '<>arg1:[e=CHEMICAL|SIMPLE

In [146]:
with open("data.pickle", "wb") as f:
    pickle.dump(data, f)
    
with open("data_dev.pickle", "wb") as f:
    pickle.dump(data_dev, f)
    
with open("data_train.pickle", "wb") as f:
    pickle.dump(data_train, f)
    
import json

def write(data, fname):
    
    with open(fname, "w") as f:
        for d in data:
            first, second = d["first"], d["second_with_arguments"]
            first_arg1 = d["first_arg1"]
            first_arg2 = d["first_arg2"]
            second_arg1 = d["second_arg1"]
            second_arg2 = d["second_arg2"]
        
            elems = [first, second, first_arg1, first_arg2, second_arg1, second_arg2]
            keys = ["first", "second", "first_arg1", "first_arg2"]
        
            f.write(json.dumps(d) + "\n")
        
            #f.write(d["first"] + "\t" + d["second"] + "\t" + d["query"] + "\t" + "-".join(d["first_arg1"])+ "\n")
            
    
write(data, "data.txt")
write(data_dev, "data_dev.txt")
write(data_train, "data_train.txt")

In [147]:
len(data_train), len(data_dev)

(41177, 7543)

In [29]:
for i in range(20):
    print(data[i]["first"], data[i]["first_arg1"], data[i]["first_arg2"])
    print(data[i]["second_with_arguments"])
    print("---------------")
    print(data[i]["query_first"])
    print(data[i]["query_second"])
    print("============================")

The structure of coronavirus 3C-like proteinase contains three domains , the first two domains form a chymotrypsin fold , which is responsible for the catalytic reaction , and <<ARG1:the extra helical domain of the enzyme>> plays an important role in controlling the associationdissociation equilibrium and regulating <<ARG2:the activity and specificity of the enzyme>> [ 6 ] . (28, 34) (46, 52)
In rotavirus infected cells an increased Akt phosphorylation depends on a direct interaction between <<ARG1:the viral nonstructural protein NSP1>> and <<ARG2:PI3 K>> .
---------------
the $enzyme <>arg1:A $[l]regulates the $enzyme <>arg2:B
the $[l]interaction between <>arg1:[e=PROTEIN]A and <>arg2:[e=PROTEIN]B
Specimens 27 and 199 reacted strongly with all Ad41 DNA probes , although types Ad2 and <<ARG1:Ad5>> emerged from <<ARG2:culture>> . (16, 16) (19, 19)
Kohl et al. [ 25 ] carried out genome pyrosequencing of bat adenovirus 2 and found there was an interspecies transmission during evolution of

In [30]:
print(len(data))

24437


In [31]:
lengths = [len(d["first"].split(" ")) + len(d["second"].split(" ")) for d in data]

In [32]:
np.mean(lengths), np.std(lengths), np.max(lengths), np.min(lengths)

(67.92299198429576, 22.057857454680867, 237, 16)

In [58]:
len([l for l in lengths if l > 250])

0